# Setup

In [1]:
#Libraries
import pandas as pd
import censusgeocode as cg
import numpy as np
import random

In [2]:
#Load in the data
addresses = pd.read_csv("data/Allegheny_County_Address_Points.csv", na_values = [""])
census_tracts = pd.read_csv("data/census_variables.csv", na_values = ['NA', '-', '**'])
income_commute = pd.read_csv("data/Alle Co income and commute census data.csv")

D:\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20,21,26,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Remove useless columns from census_tracts
census_tracts = census_tracts.drop(columns = ["Unnamed: 0", "X"])

# Join using cesus geocode

address -> GEOID

Use the result to join the datasets

In [4]:
#Filter the addresses to only Pittsburgh
addresses = addresses[addresses['MUNICIPALITY'] == 'PITTSBURGH']

In [5]:
len(addresses)

124624

In [6]:
#Define a function for taking the address census_tract geocode

def get_info(df, address_col, state_col, zip_col, city_muni_col, subset_size, seed):
    '''
    Accepts a data frame and four strings specifying columns names where
    the full address, state, zip code, and city/municipality are stored.
    Any rows with NAs in these columns are filtered. The address attributes are
    extracted, and passed to censusgeocode to retrieve GEOID, which is added
    as a new column. A subset of rows of specified size is used with the 
    specified random seed.
    '''
    # Filter out any rows with NA in address related column
    f_df = df.dropna(subset = [address_col, state_col, zip_col, city_muni_col])
    
    #Select a random subset of rows to get addresses for
    #Dont need all of them for optimization
    subset_f_df = f_df.sample(n = subset_size, random_state = seed)
    
    #Initialize an empty list to collect the GEOIDs and latlon
    geoids = []
    lats = []
    lons = []
    
    #Takes a long to run, use a counter to check progress
    total_rows = len(subset_f_df)
    rows_complete = 0
    
    #iterate on each data frame row as a tuple, get address components
    for row in subset_f_df.itertuples():
        full_add = str(getattr(row, address_col))
        state = str(getattr(row, state_col))
        zip_code = str(int(getattr(row, zip_col)))
        city = str(getattr(row, city_muni_col))
        
        #Call censusgeocode for each row to get geoid
        result = cg.address(full_add, city = city, state = state, zipcode = zip_code)
        
        #If a match isn't found, geoid is np.NaN
        if len(result) == 0:
            geoid = np.NaN
            lat = np.NaN
            lon = np.NaN
        else:
            geoid = result[0]['geographies']['2010 Census Blocks'][0]['GEOID'][0:11]
            lat = result[0]['geographies']['2010 Census Blocks'][0]['INTPTLAT']
            lon = result[0]['geographies']['2010 Census Blocks'][0]['INTPTLON']
        
            #Clean the lat lon strings
            if lat[0] == '+':
                lat = float(lat[1:])
            else:
                lat = float(lat)
            
            if lon[0] == '+':
                lon = float(lon[1:])
            else:
                lon = float(lon)
        
        #Append to the frame
        geoids.append(geoid)
        lats.append(lat)
        lons.append(lon)
        
        rows_complete += 1
        
        #Report progress
        prog = round(rows_complete/total_rows, 4)
        print(prog*100,"%")
        
        
    
    #Append to the frame and return
    subset_f_df['GEOID'] = geoids
    subset_f_df['LATITUDE'] = lats
    subset_f_df['LONGITUDE'] = lons
    
    #Print the NaN Proportion
    nan_prop = subset_f_df['GEOID'].isna().sum()/total_rows
    print('Percent NaN GEOIDs:',nan_prop*100,'%')
    
    #Return the dataframe
    return subset_f_df
        

In [7]:
#Get the frame with GEOID
addresses = get_info(addresses, "FULL_ADDRESS", "STATE", "ZIP_CODE", "MUNICIPALITY", 750, 1234)

0.13 %
0.27 %
0.4 %
0.53 %
0.67 %
0.8 %
0.9299999999999999 %
1.0699999999999998 %
1.2 %
1.3299999999999998 %
1.47 %
1.6 %
1.73 %
1.87 %
2.0 %
2.13 %
2.27 %
2.4 %
2.53 %
2.67 %
2.8000000000000003 %
2.93 %
3.0700000000000003 %
3.2 %
3.3300000000000005 %
3.47 %
3.5999999999999996 %
3.73 %
3.8699999999999997 %
4.0 %
4.130000000000001 %
4.2700000000000005 %
4.3999999999999995 %
4.53 %
4.67 %
4.8 %
4.93 %
5.07 %
5.2 %
5.33 %
5.47 %
5.6000000000000005 %
5.7299999999999995 %
5.87 %
6.0 %
6.13 %
6.2700000000000005 %
6.4 %
6.529999999999999 %
6.67 %
6.800000000000001 %
6.93 %
7.07 %
7.199999999999999 %
7.33 %
7.470000000000001 %
7.6 %
7.7299999999999995 %
7.870000000000001 %
8.0 %
8.129999999999999 %
8.27 %
8.4 %
8.53 %
8.67 %
8.799999999999999 %
8.93 %
9.07 %
9.2 %
9.33 %
9.47 %
9.6 %
9.73 %
9.87 %
10.0 %
10.13 %
10.27 %
10.4 %
10.530000000000001 %
10.67 %
10.8 %
10.93 %
11.07 %
11.200000000000001 %
11.33 %
11.469999999999999 %
11.600000000000001 %
11.73 %
11.87 %
12.0 %
12.13 %
12.27 %
12.4 %


In [10]:
# Force both GEOID columns to be int for joining
addresses['GEOID'] = addresses['GEOID'].astype(str)
income_commute['GEOID'] = income_commute['GEOID'].astype(str)

# Join the addresses result with income distribution
address_dist = addresses.merge(income_commute, how = "left", left_on = "GEOID", right_on = "GEOID")

In [11]:
# Filter rows with NAs in relevant columns
address_dist = address_dist.dropna(subset = ['GEOID', 'LATITUDE', 'LONGITUDE', 
       'Total Households', 'Less than $10,000', '$10,000 to $14,999',
       '$15,000 to $19,999', '$20,000 to $24,999', '$25,000 to $29,999',
       '$30,000 to $34,999', '$35,000 to $39,999', '$40,000 to $44,999',
       '$45,000 to $49,999', '$50,000 to $59,999', '$60,000 to $74,999',
       '$75,000 to $99,999', '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more', 'Total Workers',
       'Commute to work by car'])

In [13]:
address_dist.columns

Index(['X', 'Y', 'OBJECTID', 'FEATURE_KEY', 'ADDRESS_ID', 'PARENT_ID',
       'STREET_ID', 'DUP_STREET_ID', 'ADDRESS_TYPE', 'STATUS',
       'ADDR_NUM_PREFIX', 'ADDR_NUM', 'ADDR_NUM_SUFFIX', 'ST_PREMODIFIER',
       'ST_PREFIX', 'ST_PRETYPE', 'ST_NAME', 'ST_TYPE', 'ST_POSTMODIFIER',
       'UNIT_TYPE', 'UNIT', 'FLOOR', 'MUNICIPALITY', 'COUNTY', 'STATE',
       'ZIP_CODE', 'ZIP_CODE4', 'COMMENT', 'EDIT_DATE', 'EDIT_USER', 'SOURCE',
       'EXP_FLAG', 'FULL_ADDRESS', 'ENFORCE_VALIDATION', 'SOURCE_ID',
       'ST_SUFFIX', 'GlobalID', 'GEOID', 'LATITUDE', 'LONGITUDE',
       'Total Households', 'Less than $10,000', '$10,000 to $14,999',
       '$15,000 to $19,999', '$20,000 to $24,999', '$25,000 to $29,999',
       '$30,000 to $34,999', '$35,000 to $39,999', '$40,000 to $44,999',
       '$45,000 to $49,999', '$50,000 to $59,999', '$60,000 to $74,999',
       '$75,000 to $99,999', '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more', 'Total Worke

In [21]:
# Create functions to model income and commute probality based on census distributions
def sim_income_commute(df):
    '''
    Simulates income for a data frame based on census income bins.
    '''
    #initialize empty lists
    incomes = []
    commute_statuses = []
    
    for row in df.itertuples():

        #Get the proper attributes
        households = int(getattr(row, "_41"))
        lt_10k = int(getattr(row, "_42"))
        _10_to_14_999 = int(getattr(row, "_43"))
        _15_to_19_999 = int(getattr(row, "_44"))
        _20_to_24_999 = int(getattr(row, "_45"))
        _25_to_29_999 = int(getattr(row, "_46"))
        _30_to_34_999 = int(getattr(row, "_47"))
        _35_to_39_999 = int(getattr(row, "_48"))
        _40_to_44_999 = int(getattr(row, "_49"))
        _45_to_49_999 = int(getattr(row, "_50"))
        _50_to_59_999 = int(getattr(row, "_51"))
        _60_to_75_999 = int(getattr(row, "_52"))
        _75_to_99_999 = int(getattr(row, "_53"))
        _100_to_124_999 = int(getattr(row, "_54"))
        _125_to_149_999 = int(getattr(row, "_55"))
        _150_to_199_999 = int(getattr(row, "_56"))
        gr_200k = int(getattr(row, "_57"))
        total_workers = int(getattr(row, "_58"))
        com_by_car = int(getattr(row, "_59"))
        
        #Simulate income
        #randomly generate a number between 1 and the total households
        #See what bracket that falls into and uniformly generate the income
        samples = [i for i in range(1, households+1)]
        
        #In rare cases households will be zero, just manually set income when this happens
        if samples == []:
            income = 30000
        
        else:
            x = random.sample(samples, 1)[0]
        
            #Sampling pointer
            s = 0
        
            if 1 <= x <= lt_10k:
                income = int(random.uniform(1, 9999))
        
            s += lt_10k
            
            if s < x <= (_10_to_14_999+s):
                income = int(random.uniform(10000, 14999))
            
            s += _10_to_14_999
        
            if s < x <= _15_to_19_999+s:
                income = int(random.uniform(15000, 19999))
            
            s += _15_to_19_999
        
            if s < x <= _20_to_24_999+s:
                income = int(random.uniform(20000, 24999))
            
            s += _20_to_24_999
        
            if s < x <= _25_to_29_999+s:
                income = int(random.uniform(25000, 29999))
            
            s + _25_to_29_999
            
            if s < x <= _30_to_34_999+s:
                income = int(random.uniform(30000, 34999))
            
            s += _30_to_34_999
            
            if s < x <= _35_to_39_999+s:
                income = int(random.uniform(35000, 39999))
            
            s += _35_to_39_999
        
            if s < x <= _40_to_44_999+s:
                income = int(random.uniform(40000, 44999))
            
            s += _40_to_44_999
            
            if s < x <= _45_to_49_999+s:
                income = int(random.uniform(45000, 49999))
            
            s += _45_to_49_999
            
            if s < x <= _50_to_59_999+s:
                income = int(random.uniform(50000, 59999))
            
            s += _50_to_59_999
        
            if s < x <=  _60_to_75_999+s:
                income = int(random.uniform(60000, 74999))
            
            s += _60_to_75_999
            
            if s < x <=  _75_to_99_999+s:
                income = int(random.uniform(75000, 99999))
            
            s += _75_to_99_999
            
            if s < x <=  _100_to_124_999+s:
                income = int(random.uniform(100000, 124999))
            
            s += _100_to_124_999
            
            if s < x <=  _125_to_149_999+s:
                income = int(random.uniform(125000, 149999))
            
            s += _125_to_149_999
            
            if s < x <=  _150_to_199_999+s:
                income = int(random.uniform(150000, 199999))
            
            s += _150_to_199_999
            
            if s < x <=  gr_200k+s:
                income = int(random.uniform(200000, 250000))
        
        incomes.append(income)
        
        #Simulate commute status
        #Assume independence from income, except at the highest and lowest levels
        
        if income <= 15000:
            commute = 0
        
        elif income >= 100000:
            commute = 1
        
        else:
            try:
                prob_commute = com_by_car / total_workers
            except ZeroDivisionError:
                prob_commute = 0.5
            commute = np.random.choice([1,0], 1, p = [prob_commute, 1-prob_commute])[0]
        
        commute_statuses.append(commute)
        
        #Return the resulting lists as a tuple
    return incomes, commute_statuses

In [22]:
#Run the function on our data frame
incomes_list, commute_statuses = sim_income_commute(address_dist)

#Append to the frame
address_dist['simulated_income'] = incomes_list
address_dist['simulated_commute_car'] = commute_statuses

In [23]:
address_dist['simulated_income']

0       68456
1       61681
2       50025
3      129396
4       37848
        ...  
745    148333
746    148333
747      2407
748     52981
749     58026
Name: simulated_income, Length: 730, dtype: int64

In [24]:
address_dist.columns

Index(['X', 'Y', 'OBJECTID', 'FEATURE_KEY', 'ADDRESS_ID', 'PARENT_ID',
       'STREET_ID', 'DUP_STREET_ID', 'ADDRESS_TYPE', 'STATUS',
       'ADDR_NUM_PREFIX', 'ADDR_NUM', 'ADDR_NUM_SUFFIX', 'ST_PREMODIFIER',
       'ST_PREFIX', 'ST_PRETYPE', 'ST_NAME', 'ST_TYPE', 'ST_POSTMODIFIER',
       'UNIT_TYPE', 'UNIT', 'FLOOR', 'MUNICIPALITY', 'COUNTY', 'STATE',
       'ZIP_CODE', 'ZIP_CODE4', 'COMMENT', 'EDIT_DATE', 'EDIT_USER', 'SOURCE',
       'EXP_FLAG', 'FULL_ADDRESS', 'ENFORCE_VALIDATION', 'SOURCE_ID',
       'ST_SUFFIX', 'GlobalID', 'GEOID', 'LATITUDE', 'LONGITUDE',
       'Total Households', 'Less than $10,000', '$10,000 to $14,999',
       '$15,000 to $19,999', '$20,000 to $24,999', '$25,000 to $29,999',
       '$30,000 to $34,999', '$35,000 to $39,999', '$40,000 to $44,999',
       '$45,000 to $49,999', '$50,000 to $59,999', '$60,000 to $74,999',
       '$75,000 to $99,999', '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more', 'Total Worke

In [25]:
address_dist.head()

,X,Y,OBJECTID,FEATURE_KEY,ADDRESS_ID,PARENT_ID,STREET_ID,DUP_STREET_ID,ADDRESS_TYPE,STATUS,...,"$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",Total Workers,Commute to work by car,geometry,simulated_income,simulated_commute_car
0,1.365128e+06,406906.400219,382176,382176,382176,NaN,28547,1,Building,Active,...,310.0,151.0,144.0,183.0,228.0,2337.0,1376.0,"list(list(c(-79.923142, -79.922833, -79.922984...",68456,0
1,1.356173e+06,425711.400334,37370,37370,37370,NaN,12664,1,Situs,Active,...,152.0,92.0,87.0,44.0,26.0,1735.0,1324.0,"list(list(c(-79.96069, -79.955351, -79.951054,...",61681,1
2,1.348989e+06,406992.899921,328850,328850,328850,NaN,22039,1,Situs,Active,...,200.0,181.0,168.0,118.0,76.0,1855.0,1298.0,"list(list(c(-79.976265, -79.975817, -79.973779...",50025,0
3,1.353580e+06,415179.500129,375525,375525,375525,NaN,1308,1,Situs,Active,...,40.0,70.0,68.0,41.0,27.0,675.0,456.0,"list(list(c(-79.966005, -79.96042, -79.958853,...",129396,1
4,1.371601e+06,416042.631017,888790,557574,557574,NaN,8531,1,Building,Active,...,43.0,12.0,4.0,0.0,0.0,311.0,95.0,"list(list(c(-79.90531, -79.905264, -79.904908,...",37848,0


In [26]:
#Get the relevant columns and write to CSV
cols = ['GEOID','FULL_ADDRESS', 'MUNICIPALITY', 'COUNTY', 'STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'simulated_income', 'simulated_commute_car']
final_df = address_dist[cols]

In [27]:
final_df

,GEOID,FULL_ADDRESS,MUNICIPALITY,COUNTY,STATE,ZIP_CODE,LATITUDE,LONGITUDE,simulated_income,simulated_commute_car
0,42003140800,2331 TILBURY AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15217.0,40.429206,-79.916855,68456,0
1,42003101100,5267 KEYSTONE ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15201.0,40.480946,-79.951692,61681,1
2,42003160900,2112 SIDNEY ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15203.0,40.429154,-79.975386,50025,0
3,42003050600,817 ANAHEIM ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15219.0,40.452001,-79.959719,129396,1
4,42003130300,7280 FLEURY WAY,PITTSBURGH,ALLEGHENY COUNTY,PA,15208.0,40.455458,-79.895590,37848,0
...,...,...,...,...,...,...,...,...,...,...
745,42003191800,613 WOODBOURNE AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15226.0,40.394268,-80.023588,148333,1
746,42003191600,1322 KENBERMA AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15216.0,40.413501,-80.022508,148333,1
747,42003100500,4272 STANTON AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15201.0,40.478491,-79.943522,2407,0
748,42003191700,179 LONDON TOWNE DR,PITTSBURGH,ALLEGHENY COUNTY,PA,15226.0,40.405867,-80.009433,52981,1


In [28]:
final_df.to_csv('addresses_simulated_incomes_pitt.csv', index = False)